In [11]:
from datetime import datetime, date, time, timedelta, timezone

def midpoint_time(t1: time, t2: time) -> time:
    # Pick an arbitrary reference date
    ref = date(2000, 1, 1)

    dt1 = datetime.combine(ref, t1)
    # If t2 ≤ t1, roll t2 into the next day
    dt2_date = ref if (t2 > t1) else ref + timedelta(days=1)
    dt2 = datetime.combine(dt2_date, t2)

    mid_dt = dt1 + (dt2 - dt1) / 2
    return mid_dt.time()

In [8]:
t1 = time(23, 30)
t2 = time(23, 30)
mid_time = midpoint_time(t1, t2)
print(f"Midpoint time between {t1} and {t2} is {mid_time}")

Midpoint time between 23:30:00 and 23:30:00 is 11:30:00


In [58]:
# input data in local time zones

tz1 = 5
tz2 = 8

tz1_sleep_start_ltz = time(23, 30, tzinfo=timezone(timedelta(hours=tz1)))
tz1_sleep_stop_ltz = time(7, 30,tzinfo=timezone(timedelta(hours=tz1)))

tz2_sleep_start_ltz = time(23, 30, tzinfo=timezone(timedelta(hours=tz2)))
tz2_sleep_stop_ltz = time(7, 30, tzinfo=timezone(timedelta(hours=tz2)))

travel_start_ltz = datetime(2025, 6, 1, 4, 30, tzinfo=timezone(timedelta(hours=tz1)))
travel_stop_ltz = datetime(2025, 6, 2, 14, 30, tzinfo=timezone(timedelta(hours=tz2)))

In [ ]:
travel_start = travel_start_ltz.astimezone(timezone.utc)
travel_stop = travel_stop_ltz.astimezone(timezone.utc)

travel_window = (travel_stop, travel_stop)

print(f"Travel start: {travel_start.isoformat()}")
print(f"Travel stop:  {travel_stop.isoformat()}")

Travel start: 2025-05-31T23:30:00+00:00
Travel stop:  2025-06-02T06:30:00+00:00


In [24]:
total_travel_time = travel_stop - travel_start
print(f"Total travel time: {total_travel_time}")

Total travel time: 1 day, 7:00:00


In [26]:
start_date = travel_start.date()
print(f"Start date in UTC: {start_date.isoformat()}")

Start date in UTC: 2025-05-31


In [45]:
def sum_time_timedelta(t: time, delta: timedelta) -> time:
    # Use a mid-range date so subtraction can't underflow
    ref_date = date(2000, 1, 1)
    dt = datetime.combine(ref_date, t) + delta
    new_t = dt.time()
    return new_t.replace(tzinfo=t.tzinfo) if t.tzinfo else new_t

In [46]:
def astimezone_time(t: time, tz: timezone) -> time:
    """Convert local time to UTC."""
    ref_date = date(2000, 1, 1)
    dt = datetime.combine(ref_date, t).astimezone(tz)
    return dt.time().replace(tzinfo=tz)

In [59]:
# first CBTmin datetime object is combined from the time calculated and from the start date
CBTmin_time_start_ltz = sum_time_timedelta(tz1_sleep_stop_ltz, timedelta(hours=-3))
print(f"CBTmin start time in LTZ: {CBTmin_time_start_ltz}")
CBTmin_time_start = astimezone_time(CBTmin_time_start_ltz, timezone.utc)
print(f"CBTmin start time in UTC: {CBTmin_time_start}")

first_CBTmin = datetime.combine(start_date, tz1_sleep_start_ltz, tzinfo=timezone.utc)
print(f"First CBTmin in UTC: {first_CBTmin.isoformat()}")

CBTmin_time_dest_ltz = sum_time_timedelta(tz2_sleep_stop_ltz, timedelta(hours=-3))
print(f"CBTmin dest time in LTZ: {CBTmin_time_dest_ltz}")
CBTmin_time_dest = astimezone_time(CBTmin_time_dest_ltz, timezone.utc)
print(f"CBTmin dest time in UTC: {CBTmin_time_dest}")

CBTmin start time in LTZ: 04:30:00+05:00
CBTmin start time in UTC: 23:30:00+00:00
First CBTmin in UTC: 2025-05-31T23:30:00+00:00
CBTmin dest time in LTZ: 04:30:00+08:00
CBTmin dest time in UTC: 20:30:00+00:00


In [ ]:
def subtract_times(t1: time, t2: time):
    ref_date = date(2000, 1, 1)
    dt1 = datetime.combine(ref_date, t1)
    dt2 = datetime.combine(ref_date, t2)
    return dt1 - dt2

In [53]:
def hours_from_timedelta(td: timedelta) -> float:
    return td.total_seconds() / 3600.0

In [65]:
delta_cbtmin = subtract_times(CBTmin_time_dest, CBTmin_time_start)
print(f"Delta CBTmin: {delta_cbtmin}")
print(f"Delta CBTmin in hours: {hours_from_timedelta(delta_cbtmin)}")

Delta CBTmin: -1 day, 21:00:00
Delta CBTmin in hours: -3.0


In [91]:
if abs(hours_from_timedelta(delta_cbtmin)) < 3:
    print("CBTmin times are close enough, no adjustment needed.")

if hours_from_timedelta(delta_cbtmin) > 0:
    print("CBTmin at destination is later than at origin")
    delay = True
else:
    print("CBTmin at destination is earlier than at origin")
    delay = False
    

CBTmin at destination is earlier than at origin


In [86]:
# numbers from article, relative to CBTmin
melatonin_advance = timedelta(hours=-11.5)
melatonin_delay = timedelta(hours=4)

light_advance_window = (timedelta(hours=0), timedelta(hours=3))
light_delay_window = (timedelta(hours=-3), timedelta(hours=0))
                        
dark_advance_window = (timedelta(hours=-3), timedelta(hours=0))
dark_delay_window = (timedelta(hours=0), timedelta(hours=3))

exercise_advance_window = (timedelta(hours=0), timedelta(hours=3))
exercise_delay_window = (timedelta(hours=-3), timedelta(hours=0))

In [87]:
use_melatonin = True
use_light = True
use_exercise = True
take_melatonin_while_traveling = True
light_when_traveling = True
exercise_when_traveling = True

In [88]:
CBTmin_times = []

a=True
i=0
while a:
    i += 1
    if i == 1:
        CBTmin_times.append(first_CBTmin)
        continue
    next_CBTmin = CBTmin_times[-1] + timedelta(hours=24)
    if use_melatonin or use_light or use_exercise:
        if delay:
            next_CBTmin += timedelta(hours=1)
        else:
            next_CBTmin += timedelta(hours=-1)
    else:
        next_CBTmin += timedelta(hours=0.5)
    CBTmin_times.append(next_CBTmin)
    print(f"Next CBTmin: {next_CBTmin.isoformat()}")
    if timedelta(hours=-0.5) < subtract_times(astimezone_time(next_CBTmin.time(), timezone.utc), CBTmin_time_dest) < timedelta(hours=0.5):
        a = False

print(f"Total CBTmin times: {CBTmin_times}")

Next CBTmin: 2025-06-01T22:30:00+00:00
Next CBTmin: 2025-06-02T21:30:00+00:00
Total CBTmin times: [datetime.datetime(2025, 5, 31, 23, 30, tzinfo=datetime.timezone.utc), datetime.datetime(2025, 6, 1, 22, 30, tzinfo=datetime.timezone.utc), datetime.datetime(2025, 6, 2, 21, 30, tzinfo=datetime.timezone.utc)]


In [ ]:
melatonin_times = []
light_time_windows= []
dark_time_windows = []
exercise_time_windows = []

if use_melatonin:
    for cbt in CBTmin_times:
        if delay:
            melatonin_time = cbt + melatonin_delay
        else:
            melatonin_time = cbt + melatonin_advance
        #if take_melatonin_while_traveling:
        #    pass
        melatonin_times.append(melatonin_time)
        print(f"Melatonin time: {melatonin_time.isoformat()}")
        
        
if use_light:
    for cbt in CBTmin_times:
        if delay:
            light_time_window = (cbt + light_delay_window[0], cbt + light_delay_window[1])
            dark_time_window = (cbt + dark_delay_window[0], cbt + dark_delay_window[1])
        else:
            light_time_window = (cbt + light_advance_window[0], cbt + light_advance_window[1])
            dark_time_window = (cbt + dark_advance_window[0], cbt + dark_advance_window[1])
        #if light_when_traveling:
        #    pass
        light_time_windows.append(light_time_window)
        dark_time_windows.append(dark_time_window)
        print(f"Light time window: {light_time_window[0].isoformat()} - {light_time_window[1].isoformat()}")
        print(f"Dark time window: {dark_time_window[0].isoformat()} - {dark_time_window[1].isoformat()}")
        
if use_exercise:
    for cbt in CBTmin_times:
        if delay:
            exercise_time_window = (cbt + exercise_delay_window[0], cbt + exercise_delay_window[1])
        else:
            exercise_time_window = (cbt + exercise_advance_window[0], cbt + exercise_advance_window[1])
        #if exercise_when_traveling:
        #    pass
        exercise_time_windows.append(exercise_time_window)
        print(f"Exercise time window: {exercise_time_window[0].isoformat()} - {exercise_time_window[1].isoformat()}")

Melatonin time: 2025-05-31T12:00:00+00:00
Melatonin time: 2025-06-01T11:00:00+00:00
Melatonin time: 2025-06-02T10:00:00+00:00
Light time window: 2025-05-31T23:30:00+00:00 - 2025-06-01T02:30:00+00:00
Dark time window: 2025-05-31T20:30:00+00:00 - 2025-05-31T23:30:00+00:00
Light time window: 2025-06-01T22:30:00+00:00 - 2025-06-02T01:30:00+00:00
Dark time window: 2025-06-01T19:30:00+00:00 - 2025-06-01T22:30:00+00:00
Light time window: 2025-06-02T21:30:00+00:00 - 2025-06-03T00:30:00+00:00
Dark time window: 2025-06-02T18:30:00+00:00 - 2025-06-02T21:30:00+00:00
Exercise time window: 2025-05-31T23:30:00+00:00 - 2025-06-01T02:30:00+00:00
Exercise time window: 2025-06-01T22:30:00+00:00 - 2025-06-02T01:30:00+00:00
Exercise time window: 2025-06-02T21:30:00+00:00 - 2025-06-03T00:30:00+00:00
